In [6]:
pip install drawSvg

  Created wheel for cairocffi: filename=cairocffi-1.3.0-py3-none-any.whl size=89673 sha256=7a6e7d7a9f4c331642c1a635a0ff603fcc65c14dd67bf2786a1cb4e5658292c3
  Stored in directory: c:\users\paul\appdata\local\pip\cache\wheels\4e\ca\e1\5c8a9692a27f639a07c949044bec943f26c81cd53d3805319f
Successfully built cairocffi
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ipywidgets as widgets
import numpy as np
from ipywidgets import interact,FloatSlider,IntSlider

import drawSvg as draw
from math import *

from scipy.optimize import minimize


C:\Users\Paul\Anaconda3\lib\site-packages\drawSvg\raster.py:18: RuntimeWarning: Failed to import CairoSVG. drawSvg will be unable to output PNG or other raster image formats. See https://github.com/cduck/drawSvg#prerequisites for more details.
Original OSError: no library called "cairo-2" was found
no library called "cairo" was found
no library called "libcairo-2" was found
cannot load library 'libcairo.so.2': error 0x7e
cannot load library 'libcairo.2.dylib': error 0x7e
cannot load library 'libcairo-2.dll': error 0x7e
  warnings.warn(msg, RuntimeWarning)


In [43]:
radius = 100
canRadius = 30
width = 2 * radius + 5
height = width
d = draw.Drawing(width, height, origin='center', displayInline=False)

def intersectionLineCircle(x1,y1,x2,y2,r):
    dx = x2 - x1
    dy = y2 - y1
    dr = sqrt(dx**2+dy**2)
    deter = x1 * y2 - x2 * y1
    xpart1 = deter * dy
    xpart2 = np.sign(dy) * dx * sqrt(r**2 * dr**2 - deter**2)
    denom = dr**2
    ypart1 = -deter * dx
    ypart2 = abs(dy) * sqrt(r**2 * dr**2 - deter**2)
    ix1 = (xpart1 + xpart2) / dr**2
    ix2 = (xpart1 - xpart2) / dr**2
    iy1 = (ypart1 + ypart2) / dr**2
    iy2 = (ypart1 - ypart2) / dr**2
    return (ix1,iy1,ix2,iy2)

def polyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

def calc_area(angleEnd,angleStart,h,v,r):
    # let circle have radius 1
    sectorArea = (angleEnd - angleStart)/2
    
    # find starting point and ending point
    xStart = -sin(angleStart)
    yStart = cos(angleStart)
    xEnd = -sin(angleEnd)
    yEnd = cos(angleEnd)
    
    # find polygon areas
    centerPolyArea = polyArea([0,xStart,xEnd],[0,yStart,yEnd])
    offPolyArea = polyArea([h,xStart,xEnd],[v,yStart,yEnd])
    
    # find area
    cap = sectorArea - centerPolyArea
    area = cap + offPolyArea
    
    return area


def calc_angle(area,angleStart,h,v,r):

    def error_area(angleEnd)  :
        calc = calc_area(angleEnd,angleStart,h,v,r)
        return (calc - area)**2
    
    # guess assuming center is in middle
    guess = angleStart + area * 2
    
    res = minimize(error_area, guess, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': False})
    return res.x[0]

def cost_calc(target,endpoint,fitEquals):
    xh,yh = target[0]
    xp,yp = target[1]
    x,y = endpoint
    # average
    xa = .5 * (xh + xp)
    ya = .5 * (yh + yp)
    # difference
    xd = xh-xp
    yd = yh-yp
    # distance from x
    if fitEquals:
        d2 = (xh-x)**2 + (yh-y)**2
    else:
        d2 = (xa-x)**2 + (ya-y)**2
    
    # distance between targets
    dt2 = xd**2 + yd**2
    cost = d2 / dt2
    return cost

    
def makeCircle(h,v,r,doDisplay=True,fitEquals=False):
    
    # draw circle
    d.append(draw.Circle(0,0,radius,
                         fill="white"  ,   
                         stroke_width=2, 
                         stroke='black'))
    
    # make points
    fractions = np.array([25.2, 12.5, 2.5, 2.0, 11.2, 18.1, 14.5, 5.9, 8.2]) / 100
    areas = fractions * pi
    
    
    # starting angle
    angle = r * pi * 2
    
    # find cartesian coordinates
    y = cos(angle) * radius
    x = -sin(angle) * radius
    
    
    # collect svg regions
    points = []
    angles = []
    
    for i in range(9):
        
        # find polar coordinates
        newAngle = calc_angle(areas[i],angle,h,v,r)
        
        # find cartesian coordinates
        ny = cos(newAngle) * radius
        nx = -sin(newAngle) * radius
        
        points.append( [[x,y],[nx,ny]] )
        angles.append([angle,newAngle])
        
        angle = newAngle
    
    # draw svg regions
    
    for i in range(9):
        
        x,y = points[i][1]
        # make line from h,v to x,y
        
        
        # Draw a shape to fill with the gradient
        p = draw.Path(fill="white", 
            stroke='black',           
            stroke_width= 1,
            )
        a0 = angles[i][0]*180/pi + 90
        a1 = angles[i][1]*180/pi + 90
        p.arc(0,0,radius,a0,a1)
        p.L(h*radius, v*radius)
        p.Z()
        d.append(p)

        # emphasis on dividing lines
        d.append(draw.Lines(
            h*radius , v*radius ,
            x , y ,
            close=False,
            stroke_width= 4 if i % 3 == 2 else 1,
            fill='#eeee00',
            stroke='black'))
    
    
    # collect fitness data
    endpoints = []
    targets = []
    
    for i in range(9):
        
        angle = angles[i][0]
        x,y = points[i][1]
        
        
        # extend dividing lines
        if i % 3 == 2:
            
            # extend line and intersect with circle.

            ix1,iy1,ix2,iy2 = intersectionLineCircle(x1=x,y1=y,x2=h*radius,y2=v*radius,r=radius)
            # make line
            d.append(draw.Lines(
                ix1,iy1,
                ix2,iy2,
                close=False,
                fill='#eeee00',
                stroke_width=.5,
                stroke='grey'))
            
            # find the new endpoint
            # furthest from the current one
            d1 = (ix1 - x) ** 2 + (iy1 - y) ** 2
            d2 = (ix2 - x) ** 2 + (iy2 - y) ** 2
            if (d1 > d2):
                endpoints.append([ix1,iy1])
            else:
                endpoints.append([ix2,iy2])

        # add target zones
        if i % 3 == 1:
            halfAngle = calc_angle(0.5*areas[i],angle,h,v,r)
            
            # find cartesian coordinates
            yh = cos(halfAngle) * radius
            xh = -sin(halfAngle) * radius
            
            # draw dot at half angle
            d.append(draw.Circle(xh,yh,2,
                fill="white"  ,   
                stroke_width=.5, 
                stroke='black'))  
            
            # proportional angle
            prop = areas[i-1] / (areas[i-1]+areas[i+1])
            propAngle = calc_angle(prop*areas[i],angle,h,v,r)
            
            # find cartesian coordinates
            yp = cos(propAngle) * radius
            xp = -sin(propAngle) * radius
            
            # draw dot at half angle
            d.append(draw.Circle(xp,yp,2,
                fill="grey"  ,   
                stroke_width=.5, 
                stroke='black'))  
            
            targets.append([[xh,yh],[xp,yp]])
        
    
    # draw dot at center
    d.append(draw.Circle(0,0,2,
                         fill="black"  ,   
                         stroke_width=.5, 
                         stroke='black'))    
    
    # draw candidates
    
    # boundaries between candidate territories
    vectorAx = (-sin(angles[0][0])-h)
    vectorAy = (cos(angles[0][0])-v)

    vectorBx = (-sin(angles[3][0])-h)
    vectorBy = (cos(angles[3][0])-v)
    
    vectorCx = (-sin(angles[6][0])-h)
    vectorCy = (cos(angles[6][0])-v)
    
    
    def find_angle_between(vectorAx,vectorAy,vectorBx,vectorBy):
        # todo: make into counterclockwise angle
        nA = np.linalg.norm([vectorAx,vectorAy])
        nB = np.linalg.norm([vectorBx,vectorBy])
        return acos((vectorAx * vectorBx + vectorAy * vectorBy) / (nA * nB))
    
    # angle between boundaries
    angleOpA = find_angle_between(vectorBx,vectorBy,vectorCx,vectorCy)
    angleOpB = find_angle_between(vectorCx,vectorCy,vectorAx,vectorAy)
    angleOpC = find_angle_between(vectorAx,vectorAy,vectorBx,vectorBy)
    
    # angle from boundaries to candidates
    angleA = pi - angleOpA
    angleB = pi - angleOpB
    angleC = pi - angleOpC
    
    # start angle relative to crossing center
    startAngleReCross = atan(-vectorAx/vectorAy)
    
    # candidate coordinates
    canAngle1 = startAngleReCross + angleA
    canAngle2 = startAngleReCross + angleA + 2 * angleB
    canAngle3 = startAngleReCross + angleA + 2 * angleB + 2 * angleC
    
    
    can1x = -sin(canAngle1) * canRadius + h * radius
    can1y = cos(canAngle1) * canRadius + v * radius
    can2x = -sin(canAngle2) * canRadius + h * radius
    can2y = cos(canAngle2) * canRadius + v * radius
    can3x = -sin(canAngle3) * canRadius + h * radius
    can3y = cos(canAngle3) * canRadius + v * radius
    
    # draw candidates
    
    def drawCan(x,y,fill):
        # draw dot
        d.append(draw.Circle(x,y,2, fill=fill, stroke_width=.5, stroke='black'))    
    drawCan(can1x,can1y,"blue")
    drawCan(can2x,can2y,"red")
    drawCan(can3x,can3y,"orange")
    
    # draw circles indicating distance from center
    
    can1r = np.linalg.norm([can1x,can1y])
    can2r = np.linalg.norm([can2x,can2y])
    can3r = np.linalg.norm([can3x,can3y])
    
    def drawCanCircle(r,stroke):
        # draw circle
        d.append(draw.Circle(0,0,r, fill="none",stroke_width=.5, stroke=stroke)) 
    drawCanCircle(can1r,"blue")
    drawCanCircle(can2r,"red")
    drawCanCircle(can3r,"orange")
    
    
    d.setPixelScale(2)  # Set number of pixels per geometry unit
    if doDisplay:
        display(d)
    
    cost = cost_calc(targets[2],endpoints[0],fitEquals) + cost_calc(targets[0],endpoints[1],fitEquals) + cost_calc(targets[1],endpoints[2],fitEquals)
    return cost
    
# good values: [-.08,-.2,.01]
# good values: [-.10,-.23,.01]
# good values: [-.05,-.15,.00] # near half marks

h = widgets.FloatSlider(min=-1,max=1,step=.01,value=-.10)
v = widgets.FloatSlider(min=-1,max=1,step=.01,value=-.23)
# , orientation='vertical'

r = widgets.FloatSlider(min=0,max=1,step=.01,value=.01)
interact(makeCircle,h=h,v=v,r=r)

interactive(children=(FloatSlider(value=-0.1, description='h', max=1.0, min=-1.0, step=0.01), FloatSlider(valu…

<function __main__.makeCircle(h, v, r, doDisplay=True, fitEquals=False)>

In [45]:
# optimize makeCircle for cost

def oneCircle(params):
    h,v,r = params
    return makeCircle(h,v,r,doDisplay=False,fitEquals=False)

res = minimize(oneCircle, [-.08,-.2,.01], method='nelder-mead', options={'xatol': 1e-4, 'disp': True})

res.x










Optimization terminated successfully.
         Current function value: 0.141858
         Iterations: 42
         Function evaluations: 77


array([-0.09579822, -0.23153359,  0.01260356])

In [118]:
res.x

array([-0.05855667, -0.1518419 ,  0.01402005])

In [87]:
res.x


array([-0.09628435, -0.23140781,  0.01234321])

In [40]:
res.x


array([-0.05931815, -0.15138118,  0.01320553])